In [42]:
import os
import re
import getpass
import requests
import openai
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()


True

Будем использовать ```Function Calling``` для детерминированного ответа

In [43]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "analyze_text",
            "description": "Анализирует текст на наличие предвзятости и агрессии.",
            "parameters": {
                "type": "object",
                "properties": {
                    "text_type": {
                        "type": "array",
                        "items": {
                            "type": "string",
                            "enum": ["агрессивный", "нейтральный"]
                        },
                        "description": "Тип текста: агрессивный или нейтральный."
                    },
                    "agression_words": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "Список слов, которые указывают на агрессию."
                    },
                    "prejudgment_words": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "Список слов, которые указывают на предвзятость."
                    }, 
                    "swear_words": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "Список матерных слов и ругательств."
                    }
                },
                "required": ["text_type", "agression_words", "prejudgment_words", "swear_words"]
            }
        }
    }
]

Пример текста

In [49]:
text_to_analyze = '''
    У меня есть текст. Тебе нужно сказать, к какому классу он принадлнжит: агрессивный или нейтральный.

    Ответ нужно дать одним словом.

    Так же тебе нужно вернуть мне списки слов, которые предположительно отражают предвзятость и агрессию. Так же отдельно нужно выделить в особую группу мат

    Пример. Текст

    Мне кажется, сегодня прекрасный день. Солнце светит и все вроде бы прекрасно. НО ПОЧЕМУ ТЫ НЕ СДЕЛАЛ УРОКИ, А? Или тебе въебать хорошенько, а, опездол ебаный?

    Ответ: агрессивный. Агрессия - [НО ПОЧЕМУ ТЫ НЕ СДЕЛАЛ УРОКИ, А?] Предвзятость - [Мне кажется, вроде бы]. Мат - [въебать, опездол, ебаный]

    Теперь сделай такую оценку для входного текста:

    Этот матч был полным провалом! Как можно было так бездарно проиграть? Игроки, кажется, даже не пытались бороться за победу. Тренер, видимо, забыл, что такое тактика. Каждый пропущенный гол был как нож в спину для болельщиков. Это не футбол, а позор! Надо срочно что-то менять, иначе сезон будет окончательно потерян!

    Это просто полный крах, я расстроен! Пиздец какой-то блять, я ебал. Ебаные пидорасы на газоне!

'''

Подключаемся

In [50]:
client = OpenAI(
    api_key=os.environ['VSE_GPT_API_KEY'], 
    base_url="https://api.vsegpt.ru/v1",
)

Модель и указания

In [70]:
messages = [
    {"role": "system", "content": "Ты - большая языковая модель. Анализируй текст на предвзятость и агрессию."},
    {"role": "user", "content": text_to_analyze}
]

completion = client.chat.completions.create(
    model="google/gemini-flash-1.5",
    messages=messages,
    tools=tools, 
    tool_choice={"type": "function", "function": {"name": "analyze_text"}}, 
    temperature=0.1,
    extra_headers={"X-Title": "Colab Base Example"},  
)

Извлекаем ответ:

In [71]:
response = completion.choices[0].message

if response.tool_calls:
    for tool_call in response.tool_calls:
        if tool_call.function.name == "analyze_text":
            function_args = json.loads(tool_call.function.arguments)
            # print("Результат анализа текста:")
            print(json.dumps(function_args, indent=2, ensure_ascii=False))
else:
    print("Инструмент не был вызван.")

{
  "text_type": [
    "агрессивный"
  ],
  "prejudgment_words": [
    "\"Игроки, кажется\"",
    "\"Тренер, видимо\""
  ],
  "agression_words": [
    "\"Как можно было так бездарно проиграть?\"",
    "\"Игроки, кажется, даже не пытались бороться за победу.\"",
    "\"Каждый пропущенный гол был как нож в спину для болельщиков.\"",
    "\"Это не футбол, а позор!\"",
    "\"Надо срочно что-то менять, иначе сезон будет окончательно потерян!\"",
    "\"Это просто полный крах, я расстроен!\""
  ],
  "swear_words": [
    "\"Пиздец\"",
    "\"блять\"",
    "\"ебал\"",
    "\"Ебаные\"",
    "\"пидорасы\""
  ]
}
